In [1]:
import pandas as pd

df = pd.read_excel('../data/H1_analysis/has_cv_url.xlsx')
df

/Users/Paula_1/opt/anaconda3/envs/parenthood/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ResponseId,DE13c_1_TEXT,DE13a_Name,DE13_1_1,DE13_2_1,DE13_3_1,DE13_4_1,DE13_5_1,DE14_11_1,DE13_6_1
0,Response ID,We acknowledge your privacy considerations. Wo...,"As part of this study, we aim to gain insights...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli...","Or if you prefer, you can also share your onli..."
1,R_77h3lBm8zYTS5a1,NaN,LisetteMunzCV.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R_4pD7OrlAt0l6mjL,NaN,NaN,https://scholar.google.com/citations?user=Guhy...,NaN,NaN,NaN,NaN,NaN,NaN
3,R_7OE9SfkJJ6q9jbP,NaN,NaN,https://scholar.google.com/citations?user=BSke...,NaN,NaN,NaN,NaN,NaN,NaN
4,R_11v7OTVCxgOhPWb,NaN,ORCID ID.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3231,R_2PV7RhvMAMP0hcR,NaN,CV_MWijga.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3232,R_2WwaSOfdPYRwAwN,NaN,NaN,NaN,0000-0003-1891-7522,NaN,NaN,NaN,NaN,NaN
3233,R_2w60Oo6hzaQimqC,NaN,NaN,https://scholar.google.co.uk/citations?user=Wi...,NaN,NaN,NaN,NaN,NaN,NaN
3234,R_2iFjsLGZX3qEgu0,NaN,NaN,https://scholar.google.com/citations?hl=en&use...,https://orcid.org/0000-0002-9203-7550,https://www.researchgate.net/profile/Elif-Calik,https://www.webofscience.com/wos/woscc/basic-s...,https://www.scopus.com/authid/detail.uri?autho...,NaN,NaN


In [3]:
df = df.drop(columns=["DE13c_1_TEXT"])

In [22]:
df_data = df.iloc[1:]

available_cvs = df_data['DE13a_Name'].notna().sum()
print(f"{available_cvs} CVs")

available_gscholar = df_data['DE13_1_1'].notna().sum()
print(f"{available_gscholar} Google Scholar accounts")

available_orcid = df_data['DE13_2_1'].notna().sum()
print(f"{available_orcid} Orcids")

available_rgate = df_data['DE13_3_1'].notna().sum()
print(f"{available_rgate} Research Gate accounts")

available_wscience = df_data['DE13_4_1'].notna().sum()
print(f"{available_wscience} Web of Science accounts")

available_scopus = df_data['DE13_5_1'].notna().sum()
print(f"{available_scopus} Scopus accounts")

available_oalex = df_data['DE14_11_1'].notna().sum()
print(f"{available_oalex} OpenAlex profiles found")

available_otherurl = df_data['DE13_6_1'].notna().sum()
print(f"{available_otherurl} other URLs found")

1177 CVs
928 Google Scholar accounts
1242 Orcids
401 Research Gate accounts
103 Web of Science accounts
180 Scopus accounts
7 OpenAlex profiles found
243 other URLs found


## 1. ORCIDs

I will go through each provided datatype in decreasing order by highest number given.  
First I will clean the ORCID Ids and bring them into the format required by Open Alex' API: orcid:  
https://orcid.org/0000-0001-6187-6610. 


In [23]:
orcid_df = df_data.loc[df_data['DE13_2_1'].notna(), ['ResponseId', 'DE13_2_1']]
orcid_df

,ResponseId,DE13_2_1
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018
12,R_46yreKgPLBawn3H,0000-0002-2763-0921
13,R_2ZTGtufLu3KUHW9,0000000181844747
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496
16,R_3w5HE5P6WKA3ErS,https://orcid.org/0000-0002-8785-8063
...,...,...
3218,R_216eipKa8MqFn3z,https://orcid.org/0000-0002-6409-2632
3219,R_8FlyCLCDJMPasD4,https://orcid.org/0000-0002-6873-2967
3228,R_2X0fTWPBw20QquW,0000-0003-1528-8185
3232,R_2WwaSOfdPYRwAwN,0000-0003-1891-7522


In [24]:
failed_orcid_extractions = orcid_df[orcid_df['DE13_2_1'].isna()]
print(f"{len(failed_orcid_extractions)} failed ORCID extractions")

0 failed ORCID extractions


In [25]:
import re

def extract_canonical_orcid(value):
    value = str(value).strip()

    # Match dashed ORCID: 0000-0000-0000-0000 or ...-000X
    dashed = re.search(r'\d{4}-\d{4}-\d{4}-\d{3}[\dX]', value)
    if dashed:
        return f'https://orcid.org/{dashed.group(0)}'

    # Get raw 16-character ORCID: 15 digits + digit or X, and add dashes
    raw = re.search(r'\d{15}[\dX]', value)
    if raw:
        digits = raw.group(0)
        formatted = f'{digits[:4]}-{digits[4:8]}-{digits[8:12]}-{digits[12:]}'
        return f'https://orcid.org/{formatted}'

    # Everything else is ignored
    return None


In [26]:
orcid_df['orcid_standard'] = orcid_df['DE13_2_1'].apply(extract_canonical_orcid)
orcid_df

,ResponseId,DE13_2_1,orcid_standard
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018,https://orcid.org/0000-0001-5041-5018
12,R_46yreKgPLBawn3H,0000-0002-2763-0921,https://orcid.org/0000-0002-2763-0921
13,R_2ZTGtufLu3KUHW9,0000000181844747,https://orcid.org/0000-0001-8184-4747
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496
16,R_3w5HE5P6WKA3ErS,https://orcid.org/0000-0002-8785-8063,https://orcid.org/0000-0002-8785-8063
...,...,...,...
3218,R_216eipKa8MqFn3z,https://orcid.org/0000-0002-6409-2632,https://orcid.org/0000-0002-6409-2632
3219,R_8FlyCLCDJMPasD4,https://orcid.org/0000-0002-6873-2967,https://orcid.org/0000-0002-6873-2967
3228,R_2X0fTWPBw20QquW,0000-0003-1528-8185,https://orcid.org/0000-0003-1528-8185
3232,R_2WwaSOfdPYRwAwN,0000-0003-1891-7522,https://orcid.org/0000-0003-1891-7522


In [27]:
failed_orcid_extractions = orcid_df[orcid_df['orcid_standard'].isna()]
print(f"{len(failed_orcid_extractions)} failed ORCID extractions")

17 failed ORCID extractions


In [ ]:
# I want to see the failed entries with original inputs
failed_orcid_extractions[['ResponseId', 'DE13_2_1']]

,ResponseId,DE13_2_1
29,R_4bHw34dQzUp3iPj,000000280613629
59,R_8YXmB75YGiuDUY4,2151684
1059,R_8kIvX5bxd8oEBuW,https://orcid.org/orcid-search/search?searchQu...
1068,R_2JKPy8nLPXl4GfT,0-0002-1525-5255
1234,R_2Ev6eGbhRDAGMUA,0000-0002-8988-81
1358,R_2KJgavGjIMAE5RD,We recommend that health advertisers:
1365,R_20tMEnRIJCDG0No,https://orcid.org/orcid-search/search?searchQu...
1436,R_8bIVRT6jaNhc7Ox,2
1705,R_6flVGEkboDnwB3E,0000-0002- 2652- 772X
1795,R_2q8htRVaqOC3XNb,https://doi.org/10.1080/02680939.2022.2067594


In [29]:
orcid_df_clean = orcid_df[orcid_df['orcid_standard'].notna()]

In [30]:
orcid_df_clean

,ResponseId,DE13_2_1,orcid_standard
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018,https://orcid.org/0000-0001-5041-5018
12,R_46yreKgPLBawn3H,0000-0002-2763-0921,https://orcid.org/0000-0002-2763-0921
13,R_2ZTGtufLu3KUHW9,0000000181844747,https://orcid.org/0000-0001-8184-4747
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496
16,R_3w5HE5P6WKA3ErS,https://orcid.org/0000-0002-8785-8063,https://orcid.org/0000-0002-8785-8063
...,...,...,...
3218,R_216eipKa8MqFn3z,https://orcid.org/0000-0002-6409-2632,https://orcid.org/0000-0002-6409-2632
3219,R_8FlyCLCDJMPasD4,https://orcid.org/0000-0002-6873-2967,https://orcid.org/0000-0002-6873-2967
3228,R_2X0fTWPBw20QquW,0000-0003-1528-8185,https://orcid.org/0000-0003-1528-8185
3232,R_2WwaSOfdPYRwAwN,0000-0003-1891-7522,https://orcid.org/0000-0003-1891-7522


ORCID is done. Now, CVs.

## 2. CVs

In [31]:
cv_df = df_data.loc[df_data['DE13a_Name'].notna(), ['ResponseId', 'DE13a_Name']]
cv_df

,ResponseId,DE13a_Name
1,R_77h3lBm8zYTS5a1,LisetteMunzCV.pdf
4,R_11v7OTVCxgOhPWb,ORCID ID.txt
5,R_7NX1r2rWq9dYcBb,cvn2023.pdf
8,R_22Qq9pPVMQRrRG9,Baggio Guido- CV (April 2024)_without personal...
9,R_7eEOvE9UAAotmHQ,CV.pdf
...,...,...
3224,R_8qglyjMvsVrSFlh,CV_ROUIF S 2024.pdf
3225,R_2qWALiMolFEfkpb,curriculum_vitae_AmandaSCamara_0324.pdf
3226,R_8qEZ06rnMlHmddD,CV Sabine D Zimmermann 2023.pdf
3231,R_2PV7RhvMAMP0hcR,CV_MWijga.pdf


In [32]:
num_pdfs = cv_df['DE13a_Name'].str.endswith('.pdf', na=False).sum()
print(f"{num_pdfs} entries in DE13a_Name end with .pdf")

non_pdf_rows = cv_df[~cv_df['DE13a_Name'].str.endswith('.pdf', na=False)]
display(non_pdf_rows)

num_docs = cv_df['DE13a_Name'].str.endswith('.doc', na=False).sum()
num_docxs = cv_df['DE13a_Name'].str.endswith('.docx', na=False).sum()
print(f"{num_docs} entries end with .doc")
print(f"{num_docxs} entries end with .docx")

# Find entries not ending with .pdf, .doc, or .docx
non_standard_rows = cv_df[
    ~cv_df['DE13a_Name'].str.endswith('.pdf', na=False) &
    ~cv_df['DE13a_Name'].str.endswith('.doc', na=False) &
    ~cv_df['DE13a_Name'].str.endswith('.docx', na=False)
]
display(non_standard_rows)
print(f"{len(non_standard_rows)} entries do not end with .pdf, .doc, or .docx")

803 entries in DE13a_Name end with .pdf


,ResponseId,DE13a_Name
4,R_11v7OTVCxgOhPWb,ORCID ID.txt
20,R_2zptVVvkyegb6u0,Juris Borzovs CV akred 2021.rtf
41,R_198bUqMSHgnPIxX,Alice Reid cv 2023 3 page.doc
48,R_2gOdBDgTF157aQp,Curriculum VitaeNicolette de Keizer.docx
68,R_8yK0ZAIhgpDxtOV,Harry de Boer May 2024.docx
...,...,...
3199,R_8knQjeKjNEjw61Q,CV Tahan 2023 .docx
3200,R_8ITg6JH3UlR9sB6,CV_2024_Ian_Taylor.doc
3203,R_8q9blJt20asPo50,cv Aldaberdikyzy Aidyn.doc
3214,R_4qjqGp5sJgmewId,PittsCVphysphilwqzu.docx


92 entries end with .doc
261 entries end with .docx


,ResponseId,DE13a_Name
4,R_11v7OTVCxgOhPWb,ORCID ID.txt
20,R_2zptVVvkyegb6u0,Juris Borzovs CV akred 2021.rtf
161,R_8cjFSnWXh7XspCB,CV UL2 alias
204,R_2QAgyrZH8JTCTaK,IMG_2743.png
322,R_8RjDV5U7TDHKxa1,CV Inglese since 1999.rtf
331,R_82Lycq4JdPQMi0K,CV_Kozarzewski_Eng.DOC
491,R_8VqFhIGl8ZoEZIc,CV.pages
794,R_2oL0UsKFORDJH1Y,"CVCSIC, ASP, 2023.rtf"
1016,R_8BkXLqrekRT8WnF,Curriculum-English-short.odt
1186,R_4doRLPHDSUkdRt6,Vikrant Resume 2023 .pages


21 entries do not end with .pdf, .doc, or .docx


3 different data/ file types store 1156 of 1177 CVs provided.  
Let's check now first the other datatypes given, we continue with GoogleScholar accounts.

## 3. Google Scholar Accounts

In [33]:
gs_df = df_data.loc[df_data['DE13_1_1'].notna(), ['ResponseId', 'DE13_1_1']]
gs_df

,ResponseId,DE13_1_1
2,R_4pD7OrlAt0l6mjL,https://scholar.google.com/citations?user=Guhy...
3,R_7OE9SfkJJ6q9jbP,https://scholar.google.com/citations?user=BSke...
11,R_62ojVyfY2SdzKN0,https://scholar.google.com/scholar?hl=en&as_sd...
23,R_2B4eUokwACUsT08,https://scholar.google.com/citations?user=DCWk...
27,R_2JTY94aJonph8Qx,https://scholar.google.it/citations?user=Uyrfi...
...,...,...
3223,R_8oUd1CmnYTMewcw,https://scholar.google.com/citations?user=Q4_A...
3227,R_8PXnvStEzBAgC8m,https://scholar.google.com/citations?user=AMLe...
3230,R_2sTEG4FLmmU8kOm,https://www.webofscience.com/wos/woscc/basic-s...
3233,R_2w60Oo6hzaQimqC,https://scholar.google.co.uk/citations?user=Wi...


## 4. Research Gate Accounts

In [34]:
rg_df = df_data.loc[df_data['DE13_3_1'].notna(), ['ResponseId', 'DE13_3_1']]
rg_df

,ResponseId,DE13_3_1
15,R_1gzm4gKhyxKNWBW,https://www.researchgate.net/profile/Viktoriia...
33,R_1cVUYTnhvznwxvH,https://www.researchgate.net/profile/Vesela-Mi...
36,R_2nuEIv4Y0ddIoXF,https://www.researchgate.net/profile/Jean-Bapt...
39,R_5c1ZlaLH0DVZDQ5,https://www.researchgate.net/profile/Christian...
40,R_7j2LFKfr5BPsJEp,https://www.researchgate.net/profile/Lucia-Bel...
...,...,...
3201,R_82RJ2wuuEfAQI0T,https://www.researchgate.net/profile/Laurie-An...
3209,R_8aTvXnZQE8Gef4J,https://www.researchgate.net/profile/Sonia-Alo...
3220,R_2QycPSYQFDiOaLs,https://www.researchgate.net/profile/Najeeb-Id...
3229,R_8kN3QAWTTK2z87T,https://www.researchgate.net/profile/Joachim-H...


## 5. Web of Science

In [35]:
ws_df = df_data.loc[df_data['DE13_4_1'].notna(), ['ResponseId', 'DE13_4_1']]
ws_df

,ResponseId,DE13_4_1
11,R_62ojVyfY2SdzKN0,https://www.webofscience.com/wos/woscc/summary...
30,R_8S1zWLINCbSqU7z,https://www.webofscience.com/wos/author/record...
33,R_1cVUYTnhvznwxvH,https://www.webofscience.com/wos/author/record...
54,R_2kREO1n73i4PZbL,https://www.webofscience.com/wos/author/record...
88,R_8oAVfLbXyNOru1A,https://www.webofscience.com/wos/author/record...
...,...,...
2891,R_2KiaAyJIknq2frr,https://www.webofscience.com/wos/author/record...
2903,R_8Wk9PLFqi2ETsxr,https://www-webofscience-com.proxy-ub.rug.nl/w...
2946,R_2PXalvLF5uWODff,https://www.webofscience.com/wos/author/record...
2975,R_2TBHnWk3hdrfkom,https://www.webofscience.com/wos/author/record...


## 6. Scopus

In [38]:
scopus_df = df_data.loc[df_data['DE13_5_1'].notna(), ['ResponseId', 'DE13_5_1']]
scopus_df

,ResponseId,DE13_5_1
15,R_1gzm4gKhyxKNWBW,https://www.scopus.com/authid/detail.uri?autho...
30,R_8S1zWLINCbSqU7z,https://www.scopus.com/authid/detail.uri?autho...
33,R_1cVUYTnhvznwxvH,https://www.scopus.com/authid/detail.uri?autho...
34,R_22K1UksXA8pHVfq,https://www.scopus.com/authid/detail.uri?autho...
54,R_2kREO1n73i4PZbL,https://www.scopus.com/authid/detail.uri?autho...
...,...,...
3144,R_2tDJXr0XU8HlOCQ,https://www.scopus.com/authid/detail.uri?autho...
3156,R_2q1rjasYbfm32Tf,https://www.scopus.com/authid/detail.uri?autho...
3166,R_25SIA1HKgDT0Eyy,https://www.scopus.com/authid/detail.uri?autho...
3209,R_8aTvXnZQE8Gef4J,https://www.scopus.com/authid/detail.uri?autho...


## 7. Open Alex

In [41]:
check_other_df = df_data.loc[df_data['DE14_11_1'].notna(), ['ResponseId', 'DE14_11_1']]
check_other_df

# this is open Alex

,ResponseId,DE14_11_1
656,R_22xkV3z6PwETGMG,5004494480
1091,R_2TBugvAyOw9Ji3L,https://openalex.org/authors/a5061887578
1246,R_8EPWr5kSWlA5pbJ,https://openalex.org/works?page=1&filter=autho...
1250,R_86l4dNnNx9I9x9f,https://openalex.org/works?page=3&filter=autho...
1436,R_8bIVRT6jaNhc7Ox,0
1879,R_8CxoORRcWRV7ztD,https://explore.openalex.org/authors/A5042098918
2605,R_26eCwHR26RIk332,https://openalex.org/authors/a5030518831


Not sure why, but this column has a different Question ID despite being sorted under the right ones... it should actually be DE13_5_1 and asks for an OpenAlex URL.

## 8. Other

In [42]:
own_website_df = df_data.loc[df_data['DE13_6_1'].notna(), ['ResponseId', 'DE13_6_1']]
own_website_df

,ResponseId,DE13_6_1
7,R_8qkA83d3vwB0Tz7,https://www.jura.hhu.de/lehrstuehle-und-instit...
14,R_62GqqdSdFVyfLXa,uwe staff login zoe brennan
27,R_2JTY94aJonph8Qx,https://giuseppevizzari.github.io/cv/
29,R_4bHw34dQzUp3iPj,https://centrenorbertelias.cnrs.fr/equipes-de-...
56,R_5ZWWEKn1Iyg0OYM,https://lassev.github.io/cv.pdf
...,...,...
3170,R_8GHOtoMSgu7VABo,https://www.linkedin.com/in/adenambtman/
3186,R_2mI7ieWcIWanR9e,https://www.kth.se/profile/mardal
3188,R_2ZYeJxSnpqh8xA5,https://www.etis.ee/CV/Daisy_Volmer/eng/
3190,R_8EYyPIZF3EPJ8cj,https://www.zhaw.ch/de/ueber-uns/person/grea/


# Scraping Researchers by ORCID on Open Alex

In [43]:
import requests
import json
import time
import pandas as pd

In [57]:
def get_author_id(row):
    orcid_id = row['orcid_standard']
    url = f"https://api.openalex.org/authors/{orcid_id}"

    try:
        response = requests.get(url)
        if response.status_code == 200 and response.text.strip(): 
            data = json.loads(response.text)
            time.sleep(0.5)  
            return data
        else:
            return None
    except Exception as e:
        print(f"Error for ORCID {orcid_id}: {e}")
        return None

In [ ]:
# trying with first 5 rows 
sample_df = orcid_df_clean.head(5).copy()
sample_df["retrieved_author_data"] = sample_df.apply(get_author_id, axis=1)
sample_df

,ResponseId,DE13_2_1,orcid_standard,retrieved_author_data
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018,https://orcid.org/0000-0001-5041-5018,"{'id': 'https://openalex.org/A5079865836', 'or..."
12,R_46yreKgPLBawn3H,0000-0002-2763-0921,https://orcid.org/0000-0002-2763-0921,"{'id': 'https://openalex.org/A5061820165', 'or..."
13,R_2ZTGtufLu3KUHW9,0000000181844747,https://orcid.org/0000-0001-8184-4747,"{'id': 'https://openalex.org/A5065296382', 'or..."
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496,"{'id': 'https://openalex.org/A5087740005', 'or..."
16,R_3w5HE5P6WKA3ErS,https://orcid.org/0000-0002-8785-8063,https://orcid.org/0000-0002-8785-8063,"{'id': 'https://openalex.org/A5043649135', 'or..."


In [49]:
# what is the retrieved content?
first_entry = sample_df['retrieved_author_data'].iloc[0]
print(json.dumps(first_entry, indent=2))

{
  "id": "https://openalex.org/A5079865836",
  "orcid": "https://orcid.org/0000-0001-5041-5018",
  "display_name": "Ericka Johnson",
  "display_name_alternatives": [
    "Alice K. Johnson",
    "Ericka Johnson",
    "Austin Jerome Johnson",
    "E. Johnson",
    "A. Johnson"
  ],
  "works_count": 132,
  "cited_by_count": 1211,
  "summary_stats": {
    "2yr_mean_citedness": 5.0,
    "h_index": 21,
    "i10_index": 31
  },
  "ids": {
    "openalex": "https://openalex.org/A5079865836",
    "orcid": "https://orcid.org/0000-0001-5041-5018"
  },
  "affiliations": [
    {
      "institution": {
        "id": "https://openalex.org/I102134673",
        "ror": "https://ror.org/05ynxx418",
        "display_name": "Link\u00f6ping University",
        "country_code": "SE",
        "type": "funder",
        "lineage": [
          "https://openalex.org/I102134673"
        ]
      },
      "years": [
        2025,
        2024,
        2023,
        2022,
        2021,
        2019,
        2018,
   

In [ ]:
# what keys did we retrieve from openAlex?
print(first_entry.keys())

dict_keys(['id', 'orcid', 'display_name', 'display_name_alternatives', 'works_count', 'cited_by_count', 'summary_stats', 'ids', 'affiliations', 'last_known_institutions', 'topics', 'topic_share', 'x_concepts', 'counts_by_year', 'works_api_url', 'updated_date', 'created_date'])


In [ ]:
# now I'll do it for all ORCIDs I have
orcid_df_clean["retrieved_author_data"] = orcid_df_clean.apply(get_author_id, axis=1)
orcid_df_clean

In [107]:
orcid_df_clean

,ResponseId,DE13_2_1,orcid_standard,retrieved_author_data
6,R_2DUIfkur5fbOgZo,0000-0001-5041-5018,https://orcid.org/0000-0001-5041-5018,"{'id': 'https://openalex.org/A5079865836', 'or..."
12,R_46yreKgPLBawn3H,0000-0002-2763-0921,https://orcid.org/0000-0002-2763-0921,"{'id': 'https://openalex.org/A5061820165', 'or..."
13,R_2ZTGtufLu3KUHW9,0000000181844747,https://orcid.org/0000-0001-8184-4747,"{'id': 'https://openalex.org/A5065296382', 'or..."
15,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496,"{'id': 'https://openalex.org/A5087740005', 'or..."
16,R_3w5HE5P6WKA3ErS,https://orcid.org/0000-0002-8785-8063,https://orcid.org/0000-0002-8785-8063,"{'id': 'https://openalex.org/A5043649135', 'or..."
...,...,...,...,...
3218,R_216eipKa8MqFn3z,https://orcid.org/0000-0002-6409-2632,https://orcid.org/0000-0002-6409-2632,"{'id': 'https://openalex.org/A5044882262', 'or..."
3219,R_8FlyCLCDJMPasD4,https://orcid.org/0000-0002-6873-2967,https://orcid.org/0000-0002-6873-2967,"{'id': 'https://openalex.org/A5070800805', 'or..."
3228,R_2X0fTWPBw20QquW,0000-0003-1528-8185,https://orcid.org/0000-0003-1528-8185,"{'id': 'https://openalex.org/A5013763672', 'or..."
3232,R_2WwaSOfdPYRwAwN,0000-0003-1891-7522,https://orcid.org/0000-0003-1891-7522,"{'id': 'https://openalex.org/A5076688390', 'or..."


In [63]:
num_missing = orcid_df_clean['retrieved_author_data'].isna().sum()
print(f"{num_missing} rows in 'retrieved_author_data' are NaN or empty/null")

missing_df = orcid_df_clean[orcid_df_clean['retrieved_author_data'].isna()]
missing_df


30 rows in 'retrieved_author_data' are NaN or empty/null


,ResponseId,DE13_2_1,orcid_standard,retrieved_author_data
28,R_5EaunHkYX129nRh,0009-0009-4939-4939,https://orcid.org/0009-0009-4939-4939,None
77,R_8hfnTd8GVGqt0ZF,0000-0002-3940-7292,https://orcid.org/0000-0002-3940-7292,None
112,R_27eCdCht7JBsF9M,https://orcid.org/0000-0003-2545-6314,https://orcid.org/0000-0003-2545-6314,None
250,R_2GLscYxxbUYtxcd,0000-0002-1834-400X,https://orcid.org/0000-0002-1834-400X,None
370,R_8cqDPLMMPBd3mCq,https://orcid.org/my-orcid?orcid=0000-0001-908...,https://orcid.org/0000-0001-9081-6780,None
450,R_8S97TG6p6Cpdlff,0000-0003-0060-6240,https://orcid.org/0000-0003-0060-6240,None
546,R_8kV5gXUj974j5ZL,https://orcid.org/0000-0003-2135-0539,https://orcid.org/0000-0003-2135-0539,None
586,R_21aXrM6Up0RX58B,0000-0003-1327-8231,https://orcid.org/0000-0003-1327-8231,None
811,R_24Lqox12FZt7vMZ,0000-0001-5456-992X,https://orcid.org/0000-0001-5456-992X,None
1115,R_2Bq1COoYdgw7JZf,0000-0002-8912-2615,https://orcid.org/0000-0002-8912-2615,None


I tried to restrieve from 1225 researchers  Orcids data from Open Alex. 30 Orcids from those 1225 are not found via the Open Alex API.  
This means I have 1195 from OpenAlex found by provided Orcids.

In [ ]:
orcid_df_clean_no_missing = orcid_df_clean[~orcid_df_clean['ResponseId'].isin(missing_df['ResponseId'])]
orcid_df_clean_no_missing.reset_index(drop=True, inplace=True)
len(orcid_df_clean_no_missing)

,ResponseId,DE13_2_1,orcid_standard,retrieved_author_data
0,R_2DUIfkur5fbOgZo,0000-0001-5041-5018,https://orcid.org/0000-0001-5041-5018,"{'id': 'https://openalex.org/A5079865836', 'or..."
1,R_46yreKgPLBawn3H,0000-0002-2763-0921,https://orcid.org/0000-0002-2763-0921,"{'id': 'https://openalex.org/A5061820165', 'or..."
2,R_2ZTGtufLu3KUHW9,0000000181844747,https://orcid.org/0000-0001-8184-4747,"{'id': 'https://openalex.org/A5065296382', 'or..."
3,R_1gzm4gKhyxKNWBW,http://orcid.org/0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496,"{'id': 'https://openalex.org/A5087740005', 'or..."
4,R_3w5HE5P6WKA3ErS,https://orcid.org/0000-0002-8785-8063,https://orcid.org/0000-0002-8785-8063,"{'id': 'https://openalex.org/A5043649135', 'or..."


In [110]:
orcid_df_clean_no_missing.to_csv("firstScrape_on_oA_by_orcids.csv", index=False)

In [111]:
missing_df.to_csv("not_found_orcids_on_oA.csv", index=False)

In [112]:
df_orcid = orcid_df_clean_no_missing.copy()
df_orcid_normal = orcid_df_clean_no_missing['retrieved_author_data'].apply(pd.Series)
df_final = pd.concat([df_orcid[['ResponseId']], df_orcid_normal], axis=1)
df_final.head()

,ResponseId,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,R_2DUIfkur5fbOgZo,https://openalex.org/A5079865836,https://orcid.org/0000-0001-5041-5018,Ericka Johnson,"[Alice K. Johnson, Ericka Johnson, Austin Jero...",132,1211,"{'2yr_mean_citedness': 5.0, 'h_index': 21, 'i1...",{'openalex': 'https://openalex.org/A5079865836...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I102134673', 'ro...","[{'id': 'https://openalex.org/T11464', 'displa...","[{'id': 'https://openalex.org/T10738', 'displa...","[{'id': 'https://openalex.org/C144024400', 'wi...","[{'year': 2025, 'works_count': 3, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2025-06-16T22:26:13.308664,2023-07-21
1,R_46yreKgPLBawn3H,https://openalex.org/A5061820165,https://orcid.org/0000-0002-2763-0921,John A. Timney,"[John Timney, John Anthony Timney, John A. Tim...",55,813,"{'2yr_mean_citedness': 0.5, 'h_index': 14, 'i1...",{'openalex': 'https://openalex.org/A5061820165...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I91136226', 'ror...","[{'id': 'https://openalex.org/T11746', 'displa...","[{'id': 'https://openalex.org/T13192', 'displa...","[{'id': 'https://openalex.org/C185592680', 'wi...","[{'year': 2025, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2025-06-22T12:36:14.235761,2023-07-21
2,R_2ZTGtufLu3KUHW9,https://openalex.org/A5065296382,https://orcid.org/0000-0001-8184-4747,Özge Karaosmanoğlu,"[Ozge Karaosmanoglu, Özge Karaosmanoğlu]",30,221,"{'2yr_mean_citedness': 3.75, 'h_index': 3, 'i1...",{'openalex': 'https://openalex.org/A5065296382...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I322491407', 'ro...","[{'id': 'https://openalex.org/T10390', 'displa...","[{'id': 'https://openalex.org/T10390', 'displa...","[{'id': 'https://openalex.org/C54355233', 'wik...","[{'year': 2025, 'works_count': 6, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2025-06-16T11:33:17.534037,2023-07-21
3,R_1gzm4gKhyxKNWBW,https://openalex.org/A5087740005,https://orcid.org/0000-0002-2475-1496,Viktoriia Strilets,"[V. Y. Strilets, Viktoriia Strilets, Valerii S...",11,24,"{'2yr_mean_citedness': 1.0, 'h_index': 3, 'i10...",{'openalex': 'https://openalex.org/A5087740005...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I8765205', 'ror'...","[{'id': 'https://openalex.org/T11396', 'displa...","[{'id': 'https://openalex.org/T13608', 'displa...","[{'id': 'https://openalex.org/C41008148', 'wik...","[{'year': 2025, 'works_count': 1, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2025-06-23T02:41:38.495338,2023-07-21
4,R_3w5HE5P6WKA3ErS,https://openalex.org/A5043649135,https://orcid.org/0000-0002-8785-8063,Thomas Louf,[Thomas Louf],14,24,"{'2yr_mean_citedness': 0.6666666666666666, 'h_...",{'openalex': 'https://openalex.org/A5043649135...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210095952', 'r...","[{'id': 'https://openalex.org/T12090', 'displa...","[{'id': 'https://openalex.org/T12380', 'displa...","[{'id': 'https://openalex.org/C41008148', 'wik...","[{'year': 2025, 'works_count': 1, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2025-06-17T20:11:09.678815,2023-07-21


In [113]:
df_final.columns

Index(['ResponseId', 'id', 'orcid', 'display_name',
       'display_name_alternatives', 'works_count', 'cited_by_count',
       'summary_stats', 'ids', 'affiliations', 'last_known_institutions',
       'topics', 'topic_share', 'x_concepts', 'counts_by_year',
       'works_api_url', 'updated_date', 'created_date'],
      dtype='object')

In [114]:
# what is the content in col counts_by_year?
first_entry = df_final['counts_by_year'].iloc[0]
print(json.dumps(first_entry, indent=2))

[
  {
    "year": 2025,
    "works_count": 3,
    "cited_by_count": 33
  },
  {
    "year": 2024,
    "works_count": 2,
    "cited_by_count": 80
  },
  {
    "year": 2023,
    "works_count": 7,
    "cited_by_count": 78
  },
  {
    "year": 2022,
    "works_count": 1,
    "cited_by_count": 44
  },
  {
    "year": 2021,
    "works_count": 7,
    "cited_by_count": 52
  },
  {
    "year": 2020,
    "works_count": 3,
    "cited_by_count": 56
  },
  {
    "year": 2019,
    "works_count": 12,
    "cited_by_count": 59
  },
  {
    "year": 2018,
    "works_count": 3,
    "cited_by_count": 41
  },
  {
    "year": 2017,
    "works_count": 6,
    "cited_by_count": 57
  },
  {
    "year": 2016,
    "works_count": 13,
    "cited_by_count": 69
  },
  {
    "year": 2015,
    "works_count": 1,
    "cited_by_count": 54
  },
  {
    "year": 2014,
    "works_count": 1,
    "cited_by_count": 38
  },
  {
    "year": 2013,
    "works_count": 2,
    "cited_by_count": 62
  },
  {
    "year": 2012,
    "works_c

works_count = the number of works an author has created  
works_api_url = list of all this author's works  
summary_stats = citation metrics for this author  
last_known_institutions = author's last known institutional affiliations (all authors Works sorted by pub date, most recent one selected)  
cited_by_count = total number works that cite a work of an author
x_concepts = concepts most frequently applied to works created by this author


counts_by_year gives information abou how many works this author published, and how many times they got cited binned by year. https://docs.openalex.org/api-entities/authors/author-object. 

In [115]:
# what is the content in col topics/ topic_share? And all the other columns?
first_entry = df_final['topics'].iloc[0]
print(json.dumps(first_entry, indent=2))

[
  {
    "id": "https://openalex.org/T11464",
    "display_name": "Homelessness and Social Issues",
    "count": 12,
    "subfield": {
      "id": "https://openalex.org/subfields/3600",
      "display_name": "General Health Professions"
    },
    "field": {
      "id": "https://openalex.org/fields/36",
      "display_name": "Health Professions"
    },
    "domain": {
      "id": "https://openalex.org/domains/4",
      "display_name": "Health Sciences"
    }
  },
  {
    "id": "https://openalex.org/T11028",
    "display_name": "Social Work Education and Practice",
    "count": 10,
    "subfield": {
      "id": "https://openalex.org/subfields/3321",
      "display_name": "Public Administration"
    },
    "field": {
      "id": "https://openalex.org/fields/33",
      "display_name": "Social Sciences"
    },
    "domain": {
      "id": "https://openalex.org/domains/2",
      "display_name": "Social Sciences"
    }
  },
  {
    "id": "https://openalex.org/T13332",
    "display_name": "Hi

In [116]:
# what is the content in col topics/ topic_share? And all the other columns?
first_entry = df_final['topic_share'].iloc[0]
print(json.dumps(first_entry, indent=2))

[
  {
    "id": "https://openalex.org/T10738",
    "display_name": "Posthumanist Ethics and Activism",
    "value": 0.0002069,
    "subfield": {
      "id": "https://openalex.org/subfields/3316",
      "display_name": "Cultural Studies"
    },
    "field": {
      "id": "https://openalex.org/fields/33",
      "display_name": "Social Sciences"
    },
    "domain": {
      "id": "https://openalex.org/domains/2",
      "display_name": "Social Sciences"
    }
  },
  {
    "id": "https://openalex.org/T12320",
    "display_name": "Feminist Epistemology and Gender Studies",
    "value": 0.0001804,
    "subfield": {
      "id": "https://openalex.org/subfields/3312",
      "display_name": "Sociology and Political Science"
    },
    "field": {
      "id": "https://openalex.org/fields/33",
      "display_name": "Social Sciences"
    },
    "domain": {
      "id": "https://openalex.org/domains/2",
      "display_name": "Social Sciences"
    }
  },
  {
    "id": "https://openalex.org/T13368",
    

Topics =   
Works in OpenAlex are tagged with Topics using an automated system that takes into account the available information about the work, including title, abstract, source (journal) name, and citations. There are around 4,500 Topics. Works are assigned topics using a model that assigns scores for each topic for a work. The highest-scoring topic is that work's primary_topic. We also provide additional highly ranked topics for works, in Work.topics.  

id = the openAlex Id for this topic  
display_name = english-language label of the topic  

domain = ID & highest level in the "domain, field, subfield, topic" system, least granular. See the topics overview for more explanation and a diagram.  

field = second-highest level in the "domain, field, subfield, topic" system, second-least granular. See the topics overview for more explanation and a diagram.

subfield = third-highest level in the "domain, field, subfield, topic" system, which means it is the third-least granular. 

topic_share = https://help.openalex.org/hc/en-us/articles/27255333459991-What-is-the-difference-between-topics-and-topic-share-in-OpenAlex-entities


domain > field > subfield > topic

Each row in the 'topics' column contains a list of research topics the author has worked on, with each topic linked to a 'subfield', 'field', and 'domain', representing different levels of granularity. We use 'domain' because it's the broadest and least granular category (e.g., "Social Sciences", "Health Sciences"), which makes it easier to distinguish researchers by their overall disciplinary focus.

For each researcher, we look at all their topics (e.g., “Homelessness and Social Issues”, “Social Work Education and Practice”). Each topic has a `count` — the number of papers they’ve written in that topic. We group these topics by their associated `domain` (e.g., “Health Sciences”) and sum the counts within each domain. The domain with the highest total count is taken as the researcher’s main area of work.

In [117]:
# let's try and check out for the first row/ first researcher what would be his/her field after this implementation explained above

df_copy = df_final.copy(deep=True)
first_row_topics = df_copy['topics'].iloc[0]

domain_counts = {}

for topic in first_row_topics:
    domain = topic.get("domain", {}).get("display_name")
    count = topic.get("count", 0)
    display_name = topic.get("display_name", None)
    print(f"Topic: {display_name} | Domain: {domain} | Count: {count}")
    if domain:
        domain_counts[domain] = domain_counts.get(domain, 0) + count

print("\nTotal counts per domain:")
for domain, total in sorted(domain_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{domain}: {total}")

main_domain = max(domain_counts.items(), key=lambda x: x[1])
print(f"\nMost active domain: {main_domain[0]} (Total count: {main_domain[1]})")


Topic: Homelessness and Social Issues | Domain: Health Sciences | Count: 12
Topic: Social Work Education and Practice | Domain: Social Sciences | Count: 10
Topic: Historical Studies on Reproduction, Gender, Health, and Societal Changes | Domain: Social Sciences | Count: 9
Topic: Interprofessional Education and Collaboration | Domain: Health Sciences | Count: 8
Topic: Mental Health and Patient Involvement | Domain: Health Sciences | Count: 7
Topic: Simulation-Based Education in Healthcare | Domain: Health Sciences | Count: 7
Topic: Social and Educational Sciences | Domain: Social Sciences | Count: 7
Topic: Urban, Neighborhood, and Segregation Studies | Domain: Social Sciences | Count: 6
Topic: Sexual function and dysfunction studies | Domain: Health Sciences | Count: 5
Topic: Housing, Finance, and Neoliberalism | Domain: Social Sciences | Count: 4
Topic: Biomedical Ethics and Regulation | Domain: Health Sciences | Count: 4
Topic: Sexuality, Behavior, and Technology | Domain: Social Scie

Now we do it for the complete df_copy, we extract the domain with the highest count per researcher.

In [118]:
from collections import defaultdict

def get_main_domain_from_topics(topic_list):
    if not isinstance(topic_list, list) or len(topic_list) == 0:
        return None
    
    domain_counts = defaultdict(int)
    
    for topic in topic_list:
        domain = topic.get("domain", {}).get("display_name")
        count = topic.get("count", 0)
        if domain:
            domain_counts[domain] += count
    
    if domain_counts:
        return max(domain_counts.items(), key=lambda x: x[1])[0]
    
    return None

df_copy['main_domain'] = df_copy['topics'].apply(get_main_domain_from_topics)


In [119]:
reduced_df = df_copy[['ResponseId', 'works_count', 'cited_by_count', 'affiliations', 'last_known_institutions', 'works_api_url', 'main_domain']]
reduced_df.head()

,ResponseId,works_count,cited_by_count,affiliations,last_known_institutions,works_api_url,main_domain
0,R_2DUIfkur5fbOgZo,132,1211,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I102134673', 'ro...",https://api.openalex.org/works?filter=author.i...,Social Sciences
1,R_46yreKgPLBawn3H,55,813,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I91136226', 'ror...",https://api.openalex.org/works?filter=author.i...,Physical Sciences
2,R_2ZTGtufLu3KUHW9,30,221,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I322491407', 'ro...",https://api.openalex.org/works?filter=author.i...,Health Sciences
3,R_1gzm4gKhyxKNWBW,11,24,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I8765205', 'ror'...",https://api.openalex.org/works?filter=author.i...,Physical Sciences
4,R_3w5HE5P6WKA3ErS,14,24,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210095952', 'r...",https://api.openalex.org/works?filter=author.i...,Social Sciences


In [120]:
len(reduced_df)

1195

Continue getting Orcids from provided Scopus accounts

In [123]:
def extract_author_id(scopus_url):
    if isinstance(scopus_url, str) and "authorId=" in scopus_url:
        return scopus_url.split("authorId=")[-1].split("&")[0]
    return None

scopus_df['scopus_author_id'] = scopus_df['DE13_5_1'].apply(extract_author_id)
scopus_df

,ResponseId,DE13_5_1,scopus_author_id
15,R_1gzm4gKhyxKNWBW,https://www.scopus.com/authid/detail.uri?autho...,58787462900
30,R_8S1zWLINCbSqU7z,https://www.scopus.com/authid/detail.uri?autho...,9233278800
33,R_1cVUYTnhvznwxvH,https://www.scopus.com/authid/detail.uri?autho...,57196064572
34,R_22K1UksXA8pHVfq,https://www.scopus.com/authid/detail.uri?autho...,7003280394
54,R_2kREO1n73i4PZbL,https://www.scopus.com/authid/detail.uri?autho...,56677782000
...,...,...,...
3144,R_2tDJXr0XU8HlOCQ,https://www.scopus.com/authid/detail.uri?autho...,24477907200
3156,R_2q1rjasYbfm32Tf,https://www.scopus.com/authid/detail.uri?autho...,57169980900
3166,R_25SIA1HKgDT0Eyy,https://www.scopus.com/authid/detail.uri?autho...,57210829952
3209,R_8aTvXnZQE8Gef4J,https://www.scopus.com/authid/detail.uri?autho...,24166546200


In [124]:
API_KEY = "eb87b01c68aadbda0b3392211176df40" 
HEADERS = {"X-ELS-APIKey": API_KEY, "Accept": "application/json"}

def fetch_author_metadata(author_id):
    if not author_id:
        return None
    url = f"https://api.elsevier.com/content/author/author_id/{author_id}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    return None  

raw_jsons = []
for author_id in scopus_df["scopus_author_id"]:
    raw_jsons.append(fetch_author_metadata(author_id))
    time.sleep(0.5)  

scopus_df["scopus_raw_json"] = raw_jsons

In [126]:
scopus_df.to_csv("scopus_author_data.csv", index=False)

In [127]:
# what is the retrieved content?
first_entry = scopus_df['scopus_raw_json'].iloc[0]
print(json.dumps(first_entry, indent=2))

{
  "author-retrieval-response": [
    {
      "@status": "found",
      "@_fa": "true",
      "coredata": {
        "prism:url": "http://api.elsevier.com/content/author/author_id/58787462900",
        "dc:identifier": "AUTHOR_ID:58787462900",
        "eid": "9-s2.0-58787462900",
        "orcid": "0000-0002-2475-1496",
        "document-count": "9",
        "cited-by-count": "24",
        "citation-count": "30",
        "link": [
          {
            "@href": "https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=58787462900&origin=inward",
            "@rel": "scopus-author",
            "@_fa": "true"
          },
          {
            "@href": "http://api.elsevier.com/content/author/author_id/58787462900",
            "@rel": "self",
            "@_fa": "true"
          },
          {
            "@href": "http://api.elsevier.com/content/search/scopus?query=au-id%2858787462900%29",
            "@rel": "search",
            "@_fa": "true"
          }
        ]
   

In [129]:
# how can we get the orcid
i = 0  
scopus_df["scopus_raw_json"].iloc[i]["author-retrieval-response"][0]["coredata"]["orcid"]

'0000-0002-2475-1496'

In [130]:
def extract_orcid(entry):
    try:
        return entry["author-retrieval-response"][0]["coredata"].get("orcid")
    except (TypeError, KeyError, IndexError):
        return None

scopus_df["orcid"] = scopus_df["scopus_raw_json"].apply(extract_orcid)

In [133]:
scopus_df

,ResponseId,DE13_5_1,scopus_author_id,scopus_raw_json,orcid,orcid_standard
15,R_1gzm4gKhyxKNWBW,https://www.scopus.com/authid/detail.uri?autho...,58787462900,{'author-retrieval-response': [{'@status': 'fo...,0000-0002-2475-1496,https://orcid.org/0000-0002-2475-1496
30,R_8S1zWLINCbSqU7z,https://www.scopus.com/authid/detail.uri?autho...,9233278800,{'author-retrieval-response': [{'@status': 'fo...,0000-0003-4552-1953,https://orcid.org/0000-0003-4552-1953
33,R_1cVUYTnhvznwxvH,https://www.scopus.com/authid/detail.uri?autho...,57196064572,{'author-retrieval-response': [{'@status': 'fo...,0000-0002-5089-7552,https://orcid.org/0000-0002-5089-7552
34,R_22K1UksXA8pHVfq,https://www.scopus.com/authid/detail.uri?autho...,7003280394,{'author-retrieval-response': [{'@status': 'fo...,0000-0001-7583-9976,https://orcid.org/0000-0001-7583-9976
54,R_2kREO1n73i4PZbL,https://www.scopus.com/authid/detail.uri?autho...,56677782000,{'author-retrieval-response': [{'@status': 'fo...,0000-0002-1086-9274,https://orcid.org/0000-0002-1086-9274
...,...,...,...,...,...,...
3144,R_2tDJXr0XU8HlOCQ,https://www.scopus.com/authid/detail.uri?autho...,24477907200,{'author-retrieval-response': [{'@status': 'fo...,0000-0002-7262-6788,https://orcid.org/0000-0002-7262-6788
3156,R_2q1rjasYbfm32Tf,https://www.scopus.com/authid/detail.uri?autho...,57169980900,{'author-retrieval-response': [{'@status': 'fo...,0000-0003-1588-4294,https://orcid.org/0000-0003-1588-4294
3166,R_25SIA1HKgDT0Eyy,https://www.scopus.com/authid/detail.uri?autho...,57210829952,{'author-retrieval-response': [{'@status': 'fo...,0000-0001-6611-5298,https://orcid.org/0000-0001-6611-5298
3209,R_8aTvXnZQE8Gef4J,https://www.scopus.com/authid/detail.uri?autho...,24166546200,{'author-retrieval-response': [{'@status': 'fo...,0000-0002-3254-0365,https://orcid.org/0000-0002-3254-0365


In [132]:
# I will use the function from above where I standardized all available orcids I had already
scopus_df["orcid_standard"] = scopus_df["orcid"].apply(extract_canonical_orcid)

In [ ]:
# did it fail somewhere?
missing_df_scopus = scopus_df[scopus_df["orcid_standard"].isna()].copy()
print(f"{len(missing_df_scopus)} rows in scopus_df failed to yield a valid ORCID.")

71 rows in scopus_df failed to yield a valid ORCID.


In [136]:
missing_df_scopus.to_csv("not_found_orcids_on_scopus.csv", index=False)

In [137]:
# we remove the missing ones and create a new df with only relevant cols for now
# then we call openAlex API providing the new orcids of researchers that provided scopus Id
scopus_orcid_df = scopus_df[scopus_df["orcid_standard"].notna()].copy()
scopus_orcid_df = scopus_orcid_df[["ResponseId", "scopus_author_id", "orcid_standard"]]

scopus_orcid_df["retrieved_author_data"] = scopus_orcid_df["orcid_standard"].apply(get_author_id)
scopus_orcid_df

TypeError: string indices must be integers, not 'str'